<a href="https://colab.research.google.com/github/11muskansingh/Machine-and-Deep-Learning/blob/main/NLP_Projects/Sentiment_Analysis_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis Using Bidirectional LSTM


In [3]:
import pandas as pd
import numpy as np


In [4]:
data=pd.read_csv('Tweets.csv')

In [5]:
data.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [6]:
data['sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [7]:
#total number of rows of data and all null values

print("Total rows of data:", len(data))
print("Total null values:\n", data.isnull().sum())

Total rows of data: 27481
Total null values:
 textID           0
text             1
selected_text    1
sentiment        0
dtype: int64


In [8]:
data=data.dropna()

In [9]:
independent=data.drop('sentiment',axis=1)
dependent=data['sentiment']

In [10]:
import tensorflow as tf
tf.__version__

'2.18.0'

In [11]:
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [12]:
vocab_size=5000
messages=independent.copy()

In [13]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
print(messages.columns)

Index(['textID', 'text', 'selected_text'], dtype='object')


In [15]:
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['text'].iloc[i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [16]:
one_hot_rep=[one_hot(words,vocab_size) for words in corpus]

In [17]:
print(corpus[1])
print(one_hot_rep[1])

sooo sad miss san diego
[761, 1764, 808, 492, 2735]


In [18]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
depend_f= le.fit_transform(dependent)   # 0 = negative, 1 = neutral, 2 = positive

In [19]:
sent_length=20
embedding_vector_features=40
embedded_docs=pad_sequences(one_hot_rep,padding='pre',maxlen=sent_length)
print(embedded_docs[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0  761 1764  808  492 2735]


In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_vector_features, input_length=sent_length))
model1.add(Bidirectional(LSTM(128, return_sequences=True)))
model1.add(Bidirectional(LSTM(64)))
model1.add(Dropout(0.5))
model1.add(Dense(64, activation='relu'))
model1.add(Dense(3, activation='softmax')) # Output layer for 3 classes

model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [21]:
X1_final=np.array(embedded_docs)
y1_final=np.array(depend_f)
X1_final.shape,y1_final.shape

((27480, 20), (27480,))

In [35]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X1_final,y1_final)

In [41]:
model1.fit(train_dataset,validation_data=test_dataset,epochs=10)

Epoch 1/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 64s 172ms/step - accuracy: 0.5062 - loss: 0.9717 - val_accuracy: 0.6785 - val_loss: 0.7541
Epoch 2/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 49s 152ms/step - accuracy: 0.7022 - loss: 0.7155 - val_accuracy: 0.6838 - val_loss: 0.7474
Epoch 3/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 48s 149ms/step - accuracy: 0.7433 - loss: 0.6424 - val_accuracy: 0.6790 - val_loss: 0.7701
Epoch 4/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 48s 149ms/step - accuracy: 0.7671 - loss: 0.5989 - val_accuracy: 0.6511 - val_loss: 0.8525
Epoch 5/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 47s 147ms/step - accuracy: 0.7908 - loss: 0.5518 - val_accuracy: 0.6591 - val_loss: 0.9070
Epoch 6/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 49s 153ms/step - accuracy: 0.8146 - loss: 0.5050 - val_accuracy: 0.6440 - val_loss: 0.9525
Epoch 7/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 49s 153ms/step - accuracy: 0.8365 - loss: 0.4588 - val_accuracy: 0.6451 - val_loss: 1.0476
Epoch 8/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 48s 149ms/step - accuracy: 0.8495 - loss: 0

In [27]:
y_pred1 = np.argmax(model1.predict(X1_test), axis=1)

215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [42]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y1_test,y_pred1)

array([[1900,    0,    0],
       [2756,    0,    0],
       [2214,    0,    0]])

In [29]:
import numpy as np
unique, counts = np.unique(y1_train, return_counts=True)
print(dict(zip(unique, counts)))

{np.int64(0): np.int64(5781), np.int64(1): np.int64(8350), np.int64(2): np.int64(6479)}


In [37]:
from tensorflow.keras.utils import to_categorical

y1_train_one_hot = to_categorical(y1_train, num_classes=3)
y1_test_one_hot = to_categorical(y1_test, num_classes=3)

In [40]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((X1_train, y1_train_one_hot)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((X1_test, y1_test_one_hot)).batch(64)